In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd '/content/gdrive/MyDrive/DistilKoBiLSTM'
!pip install torch==1.6.0
!pip install torchvision==0.7.0
!pip install transformers==3.3.1
!pip install pathlib
!pip install tokenizersa

In [ ]:
import time
import datetime
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle

In [ ]:
df = pd.read_csv("dataset.csv")
df

In [ ]:
from utils import Dataset, get_teacher_output
from trainer import Distil_Trainer

vocab_size = 3000
batch_size = 64
tokenizer = BertTokenizerFast(vocab_file = "tokenizer/vocab_size_{}/vocab.txt".format(str(vocab_size)), lowercase=False, strip_accents=False)
dataset = Dataset(tokenizer, tokenizer_type = "BertTokenizerFast", batch_size = batch_size)
train_iter, test_iter, valid_iter = dataset.load_data(df)

In [ ]:
hidden_dim_list = [128, 256]
embedding_dim_list = [64, 128]
loss_rate_list = [0.1, 0.9, 0.5]
temperature_list = [1, 10, 3]
train_epoch = 30

teacher_path = "teacher_model/KoELECTRA-Small-v3/"
teacher_output = get_teacher_output(teacher_path)

for hidden_dim in hidden_dim_list:
    for embedding_dim in embedding_dim_list:
        for loss_rate in loss_rate_list:
            for temperature in temperature_list:
                print("vocab_size : ", vocab_size)
                print("loss_rate : ", loss_rate)
                print("temperature: ", temperature)
                distil_trainer = Distil_Trainer(hidden_dim = hidden_dim, embedding_dim = embedding_dim, lstm_num_layers = 1, train_epoch = train_epoch,
                                                out_put_dir = "distil_scheduler/vocab_size_{}_loss_rate_{}_temperature_{}/StepLR".format(str(vocab_size), str(int(loss_rate * 100)), temperature), tokenizer = tokenizer,
                                                teacher_output = teacher_output, loss_rate = loss_rate, temperature = temperature)
                model = distil_trainer.trainer(train_iter, valid_iter, test_iter)
                distil_trainer.tb_writer.flush()
                distil_trainer.tb_writer.close()

                del [[distil_trainer]]

# del [[train_iter, test_iter, valid_iter, model]]

In [ ]:
%matplotlib inline
%load_ext tensorboard
%tensorboard --logdir=logs

In [ ]:
from utils import count_parameters
from modeling_bilstm import BiLSTM

input_dim = 4000
hidden_dim = 256
embedding_dim = 100
model = BiLSTM(input_dim = input_dim, hidden_dim = hidden_dim, output_dim = 2, embedding_dim = embedding_dim, lstm_num_layers = 1, dropout = 0.3)
count_parameters(model)